<a href="https://colab.research.google.com/github/gsabbih6/eluvio/blob/main/eluvio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tf-nightly
!pip install tensorflow-text-nightly
!pip install tf-models-nightly

     |████████████████████████████████| 454.6MB 34kB/s 
     |████████████████████████████████| 471kB 45.1MB/s 
     |████████████████████████████████| 6.1MB 43.0MB/s 
     |████████████████████████████████| 4.2MB 43.9MB/s 
     |████████████████████████████████| 4.0MB 45.8MB/s 
     |████████████████████████████████| 1.3MB 38.1MB/s 
     |████████████████████████████████| 3.9MB 28.1MB/s 
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.37.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
  Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Successfully uninstalled grpcio-1.32.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: h5py 2.10.0
    Uninstalli

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np


print(tf.__version__)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
2.6.0-dev20210417


In [ ]:
directory_url = 'https://drive.google.com/file/d/15X00ZWBjla7qGOIW33j8865QdF89IyAk/view'
file_name = '/content/Eluvio_DS_Challenge.csv'
BERT_TFHUB_URL = "https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/4"
max_length = 128
DATASET_SIZE = 509237
batch_size=4
label_data = ['worldnews']  # for the 100GB of data it may contain of the categories.

In [ ]:
def get_bert_layer(url=BERT_TFHUB_URL):
    bert_layer = hub.KerasLayer(
        handle=url,
        trainable=True)
    return bert_layer

In [ ]:
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3")





In [ ]:
def one_hot(label):
    # npa=np.array(label) 
    print('or',label)
    def inner(a):
        return [1 if i == a else 0 for i in label_data]
    npa=tf.map_fn(inner, label, [tf.int32 for i in label_data])
    print ('npa: ',npa)
    return npa# [1 if i == label else 0 for i in label_data]

In [ ]:
def preprocess(sample, lable):
    print(sample['title'])
    tokenized_inputs = preprocessor(
        sample['title'])  # preprocessor.bert_pack_inputs([tokenized_inputs,tokenized_inputs],tf.constant(max_length))
    
    print(lable)
    label = one_hot(lable)
    print(tokenized_inputs)
    print('labe:',label)
    return tokenized_inputs, tf.reshape(label,(batch_size,1))

In [ ]:
dataset = tf.data.experimental.make_csv_dataset(
    file_name,
    # select_columns=['title'],
    select_columns=['title', 'category'],
    batch_size=batch_size, shuffle_seed=123,
    label_name="category"
)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
dataset = dataset.map(preprocess)  # preprocess each text of the dataset
train_size = int(0.9 * DATASET_SIZE)  # train batch
val_size = int(0.1 * DATASET_SIZE)  # test/val batch

ValueError: ignored

In [ ]:
# spilt dataset, shufflw ia lredy activate by default in make_csv
traindataset = dataset.take(train_size)
traindataset.batch(batch_size=batch_size)
# traindataset=traindataset.map(preprocess)
test_dataset = dataset.take(val_size)
test_dataset.batch(batch_size=batch_size)
# test_dataset=test_dataset.map(preprocess)
test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)
traindataset = traindataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# my custom model to be connected to BERT
def get_custom_model(bert_model):
    text_input = dict(
        input_word_ids=tf.keras.layers.Input(shape=(max_length,),batch_size=batch_size, dtype=tf.int32, name='input_word_ids'),
        input_mask=tf.keras.layers.Input(shape=(max_length,), batch_size=batch_size,dtype=tf.int32, name='input_mask'),
        input_type_ids=tf.keras.layers.Input(shape=(max_length,),batch_size=batch_size, dtype=tf.int32, name='input_type_ids'),
    )
    print(text_input)

    output = bert_model(text_input)

    print(output)
    l0 = tf.keras.layers.Dense(256, activation='relu')(output['pooled_output'])
    l1 = tf.keras.layers.Dense(64, activation='relu')(l0)
    l2 = tf.keras.layers.Dense(len(label_data), activation='softmax')(l1)

    network = tf.keras.Model(inputs=text_input,
                             outputs=l2)
    network.summary()
    network.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return network



In [ ]:
model = get_custom_model(get_bert_layer())

In [ ]:
# t=traindataset.take(1)
# print(t)
model.fit(traindataset, validation_data=test_dataset, epochs=1, batch_size=batch_size)

In [ ]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_' + string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_' + string])
    plt.show()

plot_graphs(model.history, 'accuracy')
plot_graphs(model.history, 'loss')
